In [ ]:
import pandas as pd
import pandas_utils as pu
import rdkit_mol_identifiers as rdkit
from rdkit.Chem import PandasTools

## Read multiple library files with inchikey column
only inchikey needed

In [ ]:
libraries = {
  "mce": r"C:\git\msn_library\data\library\mce_library_all_cleaned.tsv",
  "nih": r"C:\git\msn_library\data\nih\nih_library_new_headers_cleaned_plate7_removed.tsv",
  "nencka_mce": r"C:\git\msn_library\data\iocb_libraries\Radim_mce_complete_cleaned.tsv",
  "iocb_peptide": r"C:\git\msn_library\data\iocb_libraries\iocb_peptide_library_cleaned.tsv",
  "fdl_nmr": r"C:\git\msn_library\data\iocb_libraries\Veverka_group\FDL_NMR_lib_cleaned.tsv",
  "veverka_91": r"C:\git\msn_library\data\iocb_libraries\Veverka_group\91_1054_23_cleaned.tsv",
  "veverka_92_4": r"C:\git\msn_library\data\iocb_libraries\Veverka_group\92_4_22_cleaned.tsv",
  "veverka_92_7": r"C:\git\msn_library\data\iocb_libraries\Veverka_group\92.7_22_cleaned.tsv",
  "veverka_molport": r"C:\git\msn_library\data\iocb_libraries\Veverka_group\Molport_PACKING_DATA_cleaned.tsv",
  "veverka_packing_data_91_7": r"C:\git\msn_library\data\iocb_libraries\Veverka_group\PACKING_DATA_91.723-22_cleaned.tsv",
  "veverka_packing_data_91_1": r"C:\git\msn_library\data\iocb_libraries\Veverka_group\PACKING_DATA_91.1198-22_cleaned.tsv",
  "warth_xeno_poly": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\Warth_Vienna_Xenobiotics _PoPhe_new_headers_cleaned.tsv",
  "petras_np_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\Petras_Tübingen_Natural_Product_Library_cleaned.tsv",
  "koellensperger_mix": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\Koellensperger_compounds_cleaned.tsv",
  "weizmann": r"C:\git\msn_library\data\weizmann\WeizMassV2_cleaned.tsv",
  "milana": r"C:\git\msn_library\data\pluskal_compounds\milana_fimbriulatum_alkaloids_cleaned.tsv"
  # "targetmol_np_4320": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\L6000-Targetmol-Natural Product Library for HTS-4320cpds_script_cleaned.tsv",
  # "targetmol_np_3720": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\L6010-Targetmol-Natural Product Library-3720cpds_cleaned.tsv",
  # "selleckchem_subset": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem_subset_L5000-1w_cleaned.tsv",
  # "selleckchem_np_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem-L1400-Natural-Product-Library-96-well_new_headers_cleaned.tsv",
  # "selleckchem_phenol_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem-L1410-Natural Phenol Compound Library-96-well_new_headers_cleaned.tsv",
  # "selleckchem_terpenoid_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem-L1420-Natural Terpenoid Compound Library-96-well_new_headers_cleaned.tsv",
  # "selleckchem_organic_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem-L7600-Natural-Organic-Compound-Library-96-well_new_headers_cleaned.tsv",
  # "selleckchem_flavonoid_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem-L7700-Flavonoid-Compound-Library-96-well_new_headers_cleaned.tsv",
  # "selleckchem_alkaloid_library": r"C:\git\msn_library\data\compound_libraries\cleaned_tables\selleckchem-L7900-Alkaloid-Compound-Library-96-well_new_headers_cleaned.tsv",
  # "puretitre": r"C:\git\msn_library\data\compound_libraries\raw_data\puretitre_caithnessbiotechnologies_new_header_cleaned.tsv",
  # "analyticon_megx_all": r"C:\git\msn_library\data\compound_libraries\raw_data\MEGx_Release_2023_09_01\Analyticon_MEGx_Release_2023_09_01_All_6510_cleaned.tsv",
  # "analyticon_natx_all": r"C:\git\msn_library\data\compound_libraries\raw_data\NATx_Release_2023_09_01\Analyticon_NATx_Release_2023_09_01_All_33271_cpds_cleaned.tsv", 
  # 
}

acquired = [
  "mce", 
  "nih",
  "nencka_mce",
  "iocb_peptide",
]

collaborators = [
  "petras_np_library",
  "warth_xeno_poly",
  "koellensperger_mix",
]

In [ ]:
dfs = []
for key, value in libraries.items():
  df = pu.read_dataframe(value)[["inchikey", "monoisotopic_mass"]].copy()
  df = df[df["monoisotopic_mass"] > 114].drop(columns=["monoisotopic_mass"])
  df = df.dropna(subset="inchikey").drop_duplicates(["inchikey"]).set_index(["inchikey"])
  df[key] = True
  # if key in acquired:
  #   df["acquired"] = True
  # if key in collaborators:
  #   df["collaborators"] = True
  dfs.append(df)
  
  
merged_df = pd.concat(dfs, axis=1)
merged_df["entries"] = merged_df.count(axis=1)
merged_df["split_inchikey"] = [rdkit.split_inchikey(inchikey) for inchikey in merged_df.index]


merged_df

In [ ]:
merged_df["acquired"] = merged_df[acquired].any(axis=1)
merged_df["collaborators"] = merged_df[collaborators].any(axis=1)
merged_df["accessible"] = merged_df[["acquired", "collaborators"]].any(axis=1)
merged_df

In [ ]:
not_acquired = merged_df.loc[merged_df["acquired"] == False]
not_accessible = merged_df.loc[merged_df["accessible"] == False]

In [ ]:
not_acquired 

In [ ]:
acquired = merged_df.loc[merged_df["acquired"]]
acquired

In [ ]:
merged_df[merged_df["milana"] == True]

In [ ]:
for col in ["iocb_peptide", "targetmol_np_4320", "targetmol_np_3720", "selleckchem_subset", "selleckchem_organic_library", "selleckchem_terpenoid_library", "selleckchem_flavonoid_library", "selleckchem_alkaloid_library", "selleckchem_np_library", "nencka_mce", "puretitre", "analyticon_megx_all", "analyticon_natx_all", "petras_np_library",  "warth_xeno_poly",
            "koellensperger_mix", "fdl_nmr", "veverka_91", "veverka_92_4", "veverka_92_7", "veverka_molport", "veverka_packing_data_91_7", "veverka_packing_data_91_1"]:
  print(f"Column {col} has {not_acquired[col].sum()} True values.")


In [ ]:
for col in ["iocb_peptide", "targetmol_np_4320", "targetmol_np_3720", "selleckchem_subset", "selleckchem_organic_library", "selleckchem_terpenoid_library", "selleckchem_flavonoid_library", "selleckchem_alkaloid_library", "selleckchem_np_library", "nencka_mce", "puretitre", "analyticon_megx_all", "analyticon_natx_all", "fdl_nmr", "veverka_91", "veverka_92_4", "veverka_molport", "veverka_packing_data_91_7", "veverka_packing_data_91_1"]:
  print(f"Column {col} has {not_accessible[col].sum()} True values.")

In [ ]:
merged_df[merged_df.index == "LVVKXRQZSRUVPY"]

## Same workflow for split_inchikey

In [ ]:
dfs = []
for key, value in libraries.items():
  df = pu.read_dataframe(value)[["split_inchikey"]].copy()
  df = df.dropna(subset="split_inchikey").drop_duplicates(["split_inchikey"]).set_index(["split_inchikey"])
  df[key] = True
  # if key in acquired:
  #   df["acquired"] = True
  # if key in collaborators:
  #   df["collaborators"] = True
  dfs.append(df)


merged_df = pd.concat(dfs, axis=1)
merged_df["entries"] = merged_df.count(axis=1)


merged_df

In [ ]:
merged_df["acquired"] = merged_df[acquired].any(axis=1)
merged_df["collaborators"] = merged_df[collaborators].any(axis=1)
merged_df["accessible"] = merged_df[["acquired", "collaborators"]].any(axis=1)

not_acquired = merged_df.loc[merged_df["acquired"] == False]
not_accessible = merged_df.loc[merged_df["accessible"] == False]

In [ ]:
merged_df[merged_df["milana"] == True]

In [ ]:
for col in ["targetmol_np_4320", "targetmol_np_3720", "selleckchem_subset"]:
  print(f"Column {col} has {not_acquired[col].sum()} True values.")


In [ ]:
for col in ["targetmol_np_4320", "targetmol_np_3720", "selleckchem_subset"]:
  print(f"Column {col} has {not_accessible[col].sum()} True values.")

## Extracting unique structures in compound library

In [ ]:
library_file_cleaned = r"C:\git\msn_library\data\compound_libraries\raw_data\MEGx_Release_2023_09_01\Analyticon_MEGx_Release_2023_09_01_All_6510_cleaned.tsv"
library_file = r"C:\git\msn_library\data\compound_libraries\raw_data\MEGx_Release_2023_09_01\Analyticon_MEGx_Release_2023_09_01_All_6510.tsv"

In [ ]:
cleaned_df = pu.read_dataframe(library_file_cleaned)
lib_df = pu.read_dataframe(library_file)

In [ ]:
lib_df

In [ ]:
analyticon_megx = not_acquired.loc[not_acquired["analyticon_megx_all"] == True]
analyticon_megx

In [ ]:
inchi_set = set(analyticon_megx.index)

In [ ]:
cleaned_df["new"] = [inchikey in inchi_set for inchikey in cleaned_df["inchikey"]]
cleaned_df = cleaned_df.loc[cleaned_df["new"] == True].drop_duplicates(subset="inchikey")
cleaned_df

In [ ]:
duplicates = cleaned_df[cleaned_df.duplicated(subset=["split_inchikey"], keep=False)]
duplicates[["Compound_ID", "input_name", "compound_name", "inchikey", "split_inchikey"]]

In [ ]:
id_set = set(cleaned_df["Compound_ID"])

In [ ]:
lib_df["new"] = [id in id_set for id in lib_df["Compound_ID"]]
lib_df = lib_df.loc[lib_df["new"] == True].drop(columns="new")
lib_df

In [ ]:
pu.save_dataframe(lib_df, r"C:\git\msn_library\data\compound_libraries\raw_data\analyticon_megx_6510_subset_4127.tsv" )

In [ ]:
duplicates = lib_df[lib_df.duplicated(subset=["Compound_ID"], keep=False)]
duplicates

In [ ]:
duplicates[["ID", "input_name", "compound_name", "inchikey", "split_inchikey"]]